In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
madelon_tr = pd.read_csv("../data/madelon_train.data.txt", delimiter=' ', header=None)
madelon_tr_labels = pd.read_csv("../data/madelon_train.labels.txt", delimiter=' ', header=None)
madelon_tr_labels = madelon_tr_labels.rename(columns={0: 'labels'})
madelon_tr = pd.concat([madelon_tr, madelon_tr_labels], axis=1)
madelon_tr.drop(500, axis=1, inplace=True)
uci_df = madelon_tr.sample(440)

X_train, X_test, y_train, y_test = train_test_split(uci_df.drop('labels', axis=1), uci_df['labels'], test_size=0.25)
X_train.shape, X_test.shape

((330, 500), (110, 500))

In [3]:
!pip install tqdm

In [4]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from tqdm import tqdm

BATCH_SIZE = 500
SAMPLE_SIZE = 500
SAMPLE_PERCENT = 1

con = pg2.connect(host='34.211.227.227',dbname='postgres', user='postgres')
cur = con.cursor(cursor_factory=RealDictCursor, name='customer_cursor')
cur.execute('SELECT * FROM madelon TABLESAMPLE SYSTEM ({});'.format(SAMPLE_PERCENT))

cook_sample = []

pbar = tqdm(total=SAMPLE_SIZE//BATCH_SIZE)
while True:
    records = cur.fetchmany(size=BATCH_SIZE)

    if not records:
        break

    cook_sample += records
    pbar.update(1)

cur.close() 
con.close()
pbar.close()

cook_sample_df = pd.DataFrame(cook_sample)
cook_sample_df.shape 

4it [00:05,  1.26s/it]                       


(1943, 1002)

In [6]:
cook_sample_df.head()

,_id,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_991,feat_992,feat_993,feat_994,feat_995,feat_996,feat_997,feat_998,feat_999,target
0,85,-3.034374,0.193829,-0.542962,-0.195342,0.911961,-1.242582,0.375292,0.505649,0.712301,...,-0.718674,-0.611222,0.279672,-1.103601,0.605832,-1.109501,0.403179,1.138930,0.168399,1
1,170,0.818852,0.110386,-0.496879,-0.704824,-0.171533,-1.238078,-1.719952,-0.844725,1.318110,...,-0.026914,1.271325,1.217747,1.088598,-0.823804,-0.568238,1.688175,0.692312,-1.457718,0
2,342,-0.508119,1.084169,1.597897,0.891552,-1.737361,1.371370,-0.363354,-0.216486,-0.060162,...,-0.756684,-0.034874,-0.541712,0.850476,0.479979,-1.306054,-0.415439,2.398448,1.624010,1
3,388,-0.110053,-0.901169,-2.398464,1.667652,1.310229,-2.306286,1.462755,-0.239415,-1.176142,...,-0.273960,-0.863776,-0.498875,0.637953,1.295087,-0.060478,0.871494,0.175568,-0.205807,0
4,424,-1.355486,-0.617578,-1.614461,-2.405322,0.347963,0.861269,-0.141230,-0.912197,0.209040,...,2.934701,-0.355420,1.325917,1.382990,-0.154951,-1.212209,0.667061,0.839363,0.472666,0


In [7]:
uci_df.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,labels
1071,483,453,531,462,445,474,487,480,504,475,...,481,483,350,451,555,475,483,505,506,-1
1297,490,470,448,489,499,474,494,478,515,478,...,477,510,381,558,469,492,496,525,461,1
386,489,487,534,482,537,484,439,476,450,487,...,484,494,602,516,507,477,468,530,448,-1
307,478,486,632,464,510,464,536,477,468,479,...,474,506,432,533,521,473,498,526,477,-1
750,478,493,545,493,500,471,480,476,483,486,...,481,491,587,515,539,478,481,521,521,1


In [9]:
cook_sample_df.to_pickle('cook_sample_df.pkl')

In [10]:
uci_df.to_pickle('uci_df.pkl')